In [1]:
#https://datascience.stackexchange.com/questions/40067/confusion-matrix-three-classes-python/40068 
#confusion matrix graphics code
import itertools
import matplotlib.pyplot as plt
import numpy as np


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True Growth on Sucrose')
    plt.xlabel('Predicted Growth on Sucrose')
    plt.tight_layout()

In [2]:
#put the XgBoost rf pipeline into a function so I could plug in different datasets and predicting variables quickly
def quick_rf(filelist, predicting, dropping):
    import pandas as pd
    results_dict = {}
    df = pd.read_csv(filelist[0], index_col=0)
    if len(filelist) > 1: 
        for i in range(1, len(filelist)): 
            newdf = pd.read_csv(filelist[i], index_col=0)
            df = pd.concat([newdf, df.set_index(newdf.index)], axis=1)
        else:
            pass

    import numpy as np
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import RepeatedStratifiedKFold
    from xgboost import XGBRFClassifier
    from sklearn.metrics import confusion_matrix
    from sklearn.model_selection import train_test_split
    import xgboost as xgb
    from sklearn.utils import compute_sample_weight
    import matplotlib.pyplot as plt
    df = df.dropna(subset=[predicting])
    input_ed = df.drop(dropping, axis=1)
    ml_input = np.array(input_ed) #array of input
    output = np.array(df[predicting]) #array of output
    model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs=8)
    from sklearn.model_selection import cross_val_predict
    y_pred = cross_val_predict(model, ml_input, output, cv=10)
    tn, fp, fn, tp = confusion_matrix(output, y_pred).ravel()
    print('Precision:', tp/(tp+fp))
    model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs=8)
    xtrain, xtest, ytrain, ytest=train_test_split(ml_input, output, test_size=0.10)
    class_weights_training = compute_sample_weight('balanced', ytrain)
    model.fit(xtrain, ytrain, sample_weight=class_weights_training) 
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, ml_input, output, scoring='balanced_accuracy', cv=cv)
    print('Balanced Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)), '\n')
    header = list(input_ed.columns)
    importance_list = []
    importances = list(model.feature_importances_)
    o = 0
    m = 0
    for i in model.feature_importances_:
        results_dict[header[o]] = i
        o = o + 1
    print(max(results_dict, key = results_dict.get))

In [ ]:
#Fig 2: predict growth on substates from KEGG data
predictinglist = ['C29_D.Mannitol', 'C39_Succinate', 'D.Glucose__Fermentation', 'C32_D.Glucono.1_5.lactone', 'C12_a_a.Trehalose', 'C6_D.Xylose', 'C2_D.Galactose', 'C14_Cellobiose', 'C26_Xylitol', 'C10_Sucrose', 'C15_Salicin', 'C25_Ribitol', 'C11_Maltose', 'C40_Citrate', 'C35_D.Gluconate', 'C3_L.Sorbose', 'C20_Melezitose', 'C13_Me_a.D.Glucoside', 'O1_Cycloheximide_0.01_', 'C33_2.Keto.D.Gluconate', 'C38_DL.Lactate', 'C5_D.Ribose', 'C4_D.Glucosamine', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C24_Erythritol', 'C19_Raffinose', 'C9_L.Rhamnose', 'C22_Starch']
dropping = ['D.Glucose__Fermentation', 'D.Galactose__Fermentation', 'Maltose__Fermentation', 'Me_a.D.Glucoside__Fermentation', 'Sucrose__Fermentation', 'a_a.Trehalose__Fermentation', 'Melibiose__Fermentation', 'Lactose__Fermentation', 'Cellobiose__Fermentation', 'Melezitose__Fermentation', 'Raffinose__Fermentation', 'Inulin__Fermentation', 'Starch__Fermentation', 'D.Xylose__Fermentation', 'X_50__D.Glucose__O4_', 'X_60__D.Glucose__O5_', 'Starch_formation__M1_', 'Acid_acetic_production__M2_', 'Urea_hydrolysis__M3_', 'Diazonium_Blue_B_reaction__M4_', 'Methanol__assimilation', 'Ethanol__assimilation', 'C1_D.Glucose', 'C2_D.Galactose', 'C3_L.Sorbose', 'C4_D.Glucosamine', 'C5_D.Ribose', 'C6_D.Xylose', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C9_L.Rhamnose', 'C10_Sucrose', 'C11_Maltose', 'C12_a_a.Trehalose', 'C13_Me_a.D.Glucoside', 'C14_Cellobiose', 'C15_Salicin', 'C16_Arbutin', 'C17_Melibiose', 'C18_Lactose', 'C19_Raffinose', 'C20_Melezitose', 'C21_Inulin', 'C22_Starch', 'C23_Glycerol', 'C24_Erythritol', 'C25_Ribitol', 'C26_Xylitol', 'C27_L.Arabinitol', 'C28_D.Glucitol', 'C29_D.Mannitol', 'C30_Galactitol', 'C31_myo.Inositol', 'C32_D.Glucono.1_5.lactone', 'C33_2.Keto.D.Gluconate', 'C34_5.Keto.D.Gluconate', 'C35_D.Gluconate', 'C36_D.Glucuronate', 'C37_D.Galacturonate', 'C38_DL.Lactate', 'C39_Succinate', 'C40_Citrate', 'C43_Propane_1_2_diol', 'C44_Butane_2_3_diol', 'C45_Quinic_acid', 'C46_D.glucarate', 'C47_D.Galactonate', 'C48_Palatinose', 'C49_Levulinate', 'C50_L.Malic_acid', 'C51_L.Tartaric_acid', 'C52_D.Tartaric_acid', 'C53_meso.Tartaric_acid', 'C54_Galactaric_acid', 'C55_Uric_acid', 'C56_Gentobiose', 'C57_Ethylene_glycol', 'C58_Tween_40', 'C59_Tween_60', 'C60_Tween_80', 'N1_Nitrate', 'N2_Nitrite', 'N3_Ethylamine', 'N4_L.Lysine', 'N5_Cadaverine', 'N6_Creatine', 'N7_Creatinine', 'N8_Glucosamine', 'N9_Imidazole', 'N10_D.Tryptophan', 'N11_D.Proline', 'N12_Putrescine', 'V1_w_o_vitamins', 'V2_w_o_myo.Inositol', 'V3_w_o_Pantothenate', 'V4_w_o_Biotin', 'V5_w_o_Thiamin', 'V6_w_o_Biotin___Thiamin', 'V7_w_o_Pyridoxine', 'V8_w_o_Pyridoxine___Thiamin', 'V9_w_o_Niacin', 'V10_w_o_PABA', 'O1_Cycloheximide_0.01_', 'O2_Cycloheximide_0.1_', 'O3_Acetic_acid_1_', 'O6_10__NaCl', 'O7_16__NaCl', 'O8_Growth_at_pH_3', 'O9_Growth_at_pH_9.5', 'O10_Fluconazole', 'at_4_C', 'at_12_C', 'at_15_C', 'at_18_C', 'at_21_C', 'at_25_C', 'at_30_C', 'at_35_C', 'at_37_C', 'at_40_C', 'at_42_C', 'at_45_C']
#input for RF
filelist = ['final_KEGG_genomic_s2.csv','final_metabolic_table_s3.csv']

for predicting in predictinglist:
    print(predicting)
    quick_rf(filelist, predicting, dropping)

In [ ]:
#Fig 2: predicting growth on substrates from KEGG + interpro data
#predict growth on substates from KEGG data
predictinglist = ['C29_D.Mannitol', 'C39_Succinate', 'D.Glucose__Fermentation', 'C32_D.Glucono.1_5.lactone', 'C12_a_a.Trehalose', 'C6_D.Xylose', 'C2_D.Galactose', 'C14_Cellobiose', 'C26_Xylitol', 'C10_Sucrose', 'C15_Salicin', 'C25_Ribitol', 'C11_Maltose', 'C40_Citrate', 'C35_D.Gluconate', 'C3_L.Sorbose', 'C20_Melezitose', 'C13_Me_a.D.Glucoside', 'O1_Cycloheximide_0.01_', 'C33_2.Keto.D.Gluconate', 'C38_DL.Lactate', 'C5_D.Ribose', 'C4_D.Glucosamine', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C24_Erythritol', 'C19_Raffinose', 'C9_L.Rhamnose', 'C22_Starch']
dropping = ['D.Glucose__Fermentation', 'D.Galactose__Fermentation', 'Maltose__Fermentation', 'Me_a.D.Glucoside__Fermentation', 'Sucrose__Fermentation', 'a_a.Trehalose__Fermentation', 'Melibiose__Fermentation', 'Lactose__Fermentation', 'Cellobiose__Fermentation', 'Melezitose__Fermentation', 'Raffinose__Fermentation', 'Inulin__Fermentation', 'Starch__Fermentation', 'D.Xylose__Fermentation', 'X_50__D.Glucose__O4_', 'X_60__D.Glucose__O5_', 'Starch_formation__M1_', 'Acid_acetic_production__M2_', 'Urea_hydrolysis__M3_', 'Diazonium_Blue_B_reaction__M4_', 'Methanol__assimilation', 'Ethanol__assimilation', 'C1_D.Glucose', 'C2_D.Galactose', 'C3_L.Sorbose', 'C4_D.Glucosamine', 'C5_D.Ribose', 'C6_D.Xylose', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C9_L.Rhamnose', 'C10_Sucrose', 'C11_Maltose', 'C12_a_a.Trehalose', 'C13_Me_a.D.Glucoside', 'C14_Cellobiose', 'C15_Salicin', 'C16_Arbutin', 'C17_Melibiose', 'C18_Lactose', 'C19_Raffinose', 'C20_Melezitose', 'C21_Inulin', 'C22_Starch', 'C23_Glycerol', 'C24_Erythritol', 'C25_Ribitol', 'C26_Xylitol', 'C27_L.Arabinitol', 'C28_D.Glucitol', 'C29_D.Mannitol', 'C30_Galactitol', 'C31_myo.Inositol', 'C32_D.Glucono.1_5.lactone', 'C33_2.Keto.D.Gluconate', 'C34_5.Keto.D.Gluconate', 'C35_D.Gluconate', 'C36_D.Glucuronate', 'C37_D.Galacturonate', 'C38_DL.Lactate', 'C39_Succinate', 'C40_Citrate', 'C43_Propane_1_2_diol', 'C44_Butane_2_3_diol', 'C45_Quinic_acid', 'C46_D.glucarate', 'C47_D.Galactonate', 'C48_Palatinose', 'C49_Levulinate', 'C50_L.Malic_acid', 'C51_L.Tartaric_acid', 'C52_D.Tartaric_acid', 'C53_meso.Tartaric_acid', 'C54_Galactaric_acid', 'C55_Uric_acid', 'C56_Gentobiose', 'C57_Ethylene_glycol', 'C58_Tween_40', 'C59_Tween_60', 'C60_Tween_80', 'N1_Nitrate', 'N2_Nitrite', 'N3_Ethylamine', 'N4_L.Lysine', 'N5_Cadaverine', 'N6_Creatine', 'N7_Creatinine', 'N8_Glucosamine', 'N9_Imidazole', 'N10_D.Tryptophan', 'N11_D.Proline', 'N12_Putrescine', 'V1_w_o_vitamins', 'V2_w_o_myo.Inositol', 'V3_w_o_Pantothenate', 'V4_w_o_Biotin', 'V5_w_o_Thiamin', 'V6_w_o_Biotin___Thiamin', 'V7_w_o_Pyridoxine', 'V8_w_o_Pyridoxine___Thiamin', 'V9_w_o_Niacin', 'V10_w_o_PABA', 'O1_Cycloheximide_0.01_', 'O2_Cycloheximide_0.1_', 'O3_Acetic_acid_1_', 'O6_10__NaCl', 'O7_16__NaCl', 'O8_Growth_at_pH_3', 'O9_Growth_at_pH_9.5', 'O10_Fluconazole', 'at_4_C', 'at_12_C', 'at_15_C', 'at_18_C', 'at_21_C', 'at_25_C', 'at_30_C', 'at_35_C', 'at_37_C', 'at_40_C', 'at_42_C', 'at_45_C']
#input for RF
filelist = ['final_interpro_genomic_s1.csv', 'final_KEGG_genomic_s2.csv', 'final_metabolic_table_s3.csv']

for predicting in predictinglist:
    print(predicting)
    quick_rf(filelist, predicting, dropping)

In [ ]:
#Fig 2: predicting growth on substates by metabolic data
#certain ones of these I had to add additional dropped features manually, like galactos fermentation for galactose
predictinglist = ['C29_D.Mannitol', 'C39_Succinate', 'D.Glucose__Fermentation', 'C32_D.Glucono.1_5.lactone', 'C12_a_a.Trehalose', 'C6_D.Xylose', 'C2_D.Galactose', 'C14_Cellobiose', 'C26_Xylitol', 'C10_Sucrose', 'C15_Salicin', 'C25_Ribitol', 'C11_Maltose', 'C40_Citrate', 'C35_D.Gluconate', 'C3_L.Sorbose', 'C20_Melezitose', 'C13_Me_a.D.Glucoside', 'O1_Cycloheximide_0.01_', 'C33_2.Keto.D.Gluconate', 'C38_DL.Lactate', 'C5_D.Ribose', 'C4_D.Glucosamine', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C24_Erythritol', 'C19_Raffinose', 'C9_L.Rhamnose', 'C22_Starch']
#input for RF
filelist = ['final_metabolic_table_s3.csv']

for predicting in predictinglist:
    dropping = predicting
    print(predicting)
    quick_rf(filelist, predicting, predicting)

In [ ]:
#Fig 2: predicting growth on substates by environmental data
predictinglist = ['C29_D.Mannitol', 'C39_Succinate', 'D.Glucose__Fermentation', 'C32_D.Glucono.1_5.lactone', 'C12_a_a.Trehalose', 'C6_D.Xylose', 'C2_D.Galactose', 'C14_Cellobiose', 'C26_Xylitol', 'C10_Sucrose', 'C15_Salicin', 'C25_Ribitol', 'C11_Maltose', 'C40_Citrate', 'C35_D.Gluconate', 'C3_L.Sorbose', 'C20_Melezitose', 'C13_Me_a.D.Glucoside', 'O1_Cycloheximide_0.01_', 'C33_2.Keto.D.Gluconate', 'C38_DL.Lactate', 'C5_D.Ribose', 'C4_D.Glucosamine', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C24_Erythritol', 'C19_Raffinose', 'C9_L.Rhamnose', 'C22_Starch']
#input for RF
filelist = ['final_env_table_s4.csv', 'final_metabolic_table_s3.csv']
dropping = ['D.Glucose__Fermentation', 'D.Galactose__Fermentation', 'Maltose__Fermentation', 'Me_a.D.Glucoside__Fermentation', 'Sucrose__Fermentation', 'a_a.Trehalose__Fermentation', 'Melibiose__Fermentation', 'Lactose__Fermentation', 'Cellobiose__Fermentation', 'Melezitose__Fermentation', 'Raffinose__Fermentation', 'Inulin__Fermentation', 'Starch__Fermentation', 'D.Xylose__Fermentation', 'X_50__D.Glucose__O4_', 'X_60__D.Glucose__O5_', 'Starch_formation__M1_', 'Acid_acetic_production__M2_', 'Urea_hydrolysis__M3_', 'Diazonium_Blue_B_reaction__M4_', 'Methanol__assimilation', 'Ethanol__assimilation', 'C1_D.Glucose', 'C2_D.Galactose', 'C3_L.Sorbose', 'C4_D.Glucosamine', 'C5_D.Ribose', 'C6_D.Xylose', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C9_L.Rhamnose', 'C10_Sucrose', 'C11_Maltose', 'C12_a_a.Trehalose', 'C13_Me_a.D.Glucoside', 'C14_Cellobiose', 'C15_Salicin', 'C16_Arbutin', 'C17_Melibiose', 'C18_Lactose', 'C19_Raffinose', 'C20_Melezitose', 'C21_Inulin', 'C22_Starch', 'C23_Glycerol', 'C24_Erythritol', 'C25_Ribitol', 'C26_Xylitol', 'C27_L.Arabinitol', 'C28_D.Glucitol', 'C29_D.Mannitol', 'C30_Galactitol', 'C31_myo.Inositol', 'C32_D.Glucono.1_5.lactone', 'C33_2.Keto.D.Gluconate', 'C34_5.Keto.D.Gluconate', 'C35_D.Gluconate', 'C36_D.Glucuronate', 'C37_D.Galacturonate', 'C38_DL.Lactate', 'C39_Succinate', 'C40_Citrate', 'C43_Propane_1_2_diol', 'C44_Butane_2_3_diol', 'C45_Quinic_acid', 'C46_D.glucarate', 'C47_D.Galactonate', 'C48_Palatinose', 'C49_Levulinate', 'C50_L.Malic_acid', 'C51_L.Tartaric_acid', 'C52_D.Tartaric_acid', 'C53_meso.Tartaric_acid', 'C54_Galactaric_acid', 'C55_Uric_acid', 'C56_Gentobiose', 'C57_Ethylene_glycol', 'C58_Tween_40', 'C59_Tween_60', 'C60_Tween_80', 'N1_Nitrate', 'N2_Nitrite', 'N3_Ethylamine', 'N4_L.Lysine', 'N5_Cadaverine', 'N6_Creatine', 'N7_Creatinine', 'N8_Glucosamine', 'N9_Imidazole', 'N10_D.Tryptophan', 'N11_D.Proline', 'N12_Putrescine', 'V1_w_o_vitamins', 'V2_w_o_myo.Inositol', 'V3_w_o_Pantothenate', 'V4_w_o_Biotin', 'V5_w_o_Thiamin', 'V6_w_o_Biotin___Thiamin', 'V7_w_o_Pyridoxine', 'V8_w_o_Pyridoxine___Thiamin', 'V9_w_o_Niacin', 'V10_w_o_PABA', 'O1_Cycloheximide_0.01_', 'O2_Cycloheximide_0.1_', 'O3_Acetic_acid_1_', 'O6_10__NaCl', 'O7_16__NaCl', 'O8_Growth_at_pH_3', 'O9_Growth_at_pH_9.5', 'O10_Fluconazole', 'at_4_C', 'at_12_C', 'at_15_C', 'at_18_C', 'at_21_C', 'at_25_C', 'at_30_C', 'at_35_C', 'at_37_C', 'at_40_C', 'at_42_C', 'at_45_C']


for predicting in predictinglist:
    print(predicting)
    randomsample2(filelist, predicting, dropping)

In [ ]:
#Figure 3: Predicting growth on galactose from KEGG Orthology data--repeated for Xylose and Sucrose too
import pandas as pd

#format input
input1= 'final_KEGG_genomic_s2.csv'
input2='final_metabolic_table_s3.csv'

df1 = pd.read_csv(input1, index_col=0)
df2 = pd.read_csv(input2, index_col=0)
df = pd.concat([df1, df2.set_index(df1.index)], axis=1)
#df = pd.read_csv(input2, index_col=0)


#import modules needed
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBRFClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.utils import compute_sample_weight
import matplotlib.pyplot as plt
from collections import Counter

#define what we want to predict out of the dataset
predicting = 'C2_D.Galactose'
#drop NAs (species we didn't measure specialism/generalism) from the dataset
df = df.dropna(subset=[predicting])

#decide what other data you want to drop from the input, here I am dropping the predicted value
dropping = ['D.Glucose__Fermentation', 'D.Galactose__Fermentation', 'Maltose__Fermentation', 'Me_a.D.Glucoside__Fermentation', 'Sucrose__Fermentation', 'a_a.Trehalose__Fermentation', 'Melibiose__Fermentation', 'Lactose__Fermentation', 'Cellobiose__Fermentation', 'Melezitose__Fermentation', 'Raffinose__Fermentation', 'Inulin__Fermentation', 'Starch__Fermentation', 'D.Xylose__Fermentation', 'X_50__D.Glucose__O4_', 'X_60__D.Glucose__O5_', 'Starch_formation__M1_', 'Acid_acetic_production__M2_', 'Urea_hydrolysis__M3_', 'Diazonium_Blue_B_reaction__M4_', 'Methanol__assimilation', 'Ethanol__assimilation', 'C1_D.Glucose', 'C2_D.Galactose', 'C3_L.Sorbose', 'C4_D.Glucosamine', 'C5_D.Ribose', 'C6_D.Xylose', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C9_L.Rhamnose', 'C10_Sucrose', 'C11_Maltose', 'C12_a_a.Trehalose', 'C13_Me_a.D.Glucoside', 'C14_Cellobiose', 'C15_Salicin', 'C16_Arbutin', 'C17_Melibiose', 'C18_Lactose', 'C19_Raffinose', 'C20_Melezitose', 'C21_Inulin', 'C22_Starch', 'C23_Glycerol', 'C24_Erythritol', 'C25_Ribitol', 'C26_Xylitol', 'C27_L.Arabinitol', 'C28_D.Glucitol', 'C29_D.Mannitol', 'C30_Galactitol', 'C31_myo.Inositol', 'C32_D.Glucono.1_5.lactone', 'C33_2.Keto.D.Gluconate', 'C34_5.Keto.D.Gluconate', 'C35_D.Gluconate', 'C36_D.Glucuronate', 'C37_D.Galacturonate', 'C38_DL.Lactate', 'C39_Succinate', 'C40_Citrate', 'C43_Propane_1_2_diol', 'C44_Butane_2_3_diol', 'C45_Quinic_acid', 'C46_D.glucarate', 'C47_D.Galactonate', 'C48_Palatinose', 'C49_Levulinate', 'C50_L.Malic_acid', 'C51_L.Tartaric_acid', 'C52_D.Tartaric_acid', 'C53_meso.Tartaric_acid', 'C54_Galactaric_acid', 'C55_Uric_acid', 'C56_Gentobiose', 'C57_Ethylene_glycol', 'C58_Tween_40', 'C59_Tween_60', 'C60_Tween_80', 'N1_Nitrate', 'N2_Nitrite', 'N3_Ethylamine', 'N4_L.Lysine', 'N5_Cadaverine', 'N6_Creatine', 'N7_Creatinine', 'N8_Glucosamine', 'N9_Imidazole', 'N10_D.Tryptophan', 'N11_D.Proline', 'N12_Putrescine', 'V1_w_o_vitamins', 'V2_w_o_myo.Inositol', 'V3_w_o_Pantothenate', 'V4_w_o_Biotin', 'V5_w_o_Thiamin', 'V6_w_o_Biotin___Thiamin', 'V7_w_o_Pyridoxine', 'V8_w_o_Pyridoxine___Thiamin', 'V9_w_o_Niacin', 'V10_w_o_PABA', 'O1_Cycloheximide_0.01_', 'O2_Cycloheximide_0.1_', 'O3_Acetic_acid_1_', 'O6_10__NaCl', 'O7_16__NaCl', 'O8_Growth_at_pH_3', 'O9_Growth_at_pH_9.5', 'O10_Fluconazole', 'at_4_C', 'at_12_C', 'at_15_C', 'at_18_C', 'at_21_C', 'at_25_C', 'at_30_C', 'at_35_C', 'at_37_C', 'at_40_C', 'at_42_C', 'at_45_C']
input_ed = df.drop(dropping, axis=1)
ml_input = np.array(input_ed) #array of input
output = np.array(df[predicting]) #array of output
print(output)

#build your model 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)

#use cross_val_predict to test the model and build the big confusion matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(model, ml_input, output, cv=10)
conf_mat = confusion_matrix(output, y_pred)
print(conf_mat)
plt.rcParams["figure.figsize"] = (3,3)
plt.figure()
plot_confusion_matrix(conf_mat, classes=['0','1'],
                     title='Confusion matrix')
plt.savefig("Confusion_Matrix.png", dpi=200)


#look specifically at which species are incorrectly and correctly predicted
species_list = list(df.index)
print('Incorrectly IDed  True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] != y_pred[i]:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
print('Correctly IDed   True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] == 1 and y_pred[i] == 1:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
    

header = list(input_ed.columns)

# define the model again to re-test on a held out dataset, usually have  subsample=0.9, colsample_bynode=0.8, 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
xtrain, xtest, ytrain, ytest=train_test_split(ml_input, output, test_size=0.10)
class_weights_training = compute_sample_weight('balanced', ytrain)
#sample_weight=class_weights_training
#model.fit(xtrain, ytrain) 
model.fit(xtrain, ytrain, sample_weight=class_weights_training) 

print('Mini_test')
ypred = model.predict(xtest)
cm = confusion_matrix(ytest,ypred)
total = sum(cm[0]) + sum(cm[1])
mis = cm[0][1] + cm[1][0]
print('Misclassification rate:', mis/total)

print(cm)

plt.figure()
plot_confusion_matrix(cm, classes=['0','1'],
                     title='Confusion matrix, without normalization')


#rebuild model yet again for the final big cross validation accuracy metrics with RepeatedStratifiedKFold
classes_weights = compute_sample_weight('balanced', output)
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
sample_weight=classes_weights
model.fit(ml_input, output, sample_weight=classes_weights)
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, ml_input, output, scoring='accuracy', cv=cv)
print(n_scores)
# report performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)), '\n')


#report feature importances in order of most important to least
importance_list = []
importances = list(model.feature_importances_)
o = 0
m = 0
top10 = []
for i in model.feature_importances_:
    importance_list.append([i, header[o]])
    o = o + 1
for i in importance_list:
    if i[0] == max(importances):
        print(i[1], '\t', i[0])
        if m < 10:
            top10.append(i[1])
            m = m + 1
            importances.remove(i[0])
        else:
            importances.remove(i[0])
    else:
        importance_list.append(i)

In [ ]:
#Figure 3: Predicting growth on galactose from metabolic data--repeated for sucrose and xylose too
import pandas as pd

#format input
input1='final_metabolic_table_s3.csv'

#df1 = pd.read_csv(input1, index_col=0)
#df2 = pd.read_csv(input2, index_col=0)
#df = pd.concat([df1, df2.set_index(df1.index)], axis=1)
df = pd.read_csv(input1, index_col=0)


#import modules needed
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBRFClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.utils import compute_sample_weight
import matplotlib.pyplot as plt
from collections import Counter

#define what we want to predict out of the dataset
predicting = 'C2_D.Galactose'
#drop NAs (species we didn't measure specialism/generalism) from the dataset
df = df.dropna(subset=[predicting])

#decide what other data you want to drop from the input, here I am dropping the predicted value
dropping = ['C2_D.Galactose', 'D.Galactose__Fermentation']
input_ed = df.drop(dropping, axis=1)
ml_input = np.array(input_ed) #array of input
output = np.array(df[predicting]) #array of output
print(output)

#build your model 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)

#use cross_val_predict to test the model and build the big confusion matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(model, ml_input, output, cv=10)
conf_mat = confusion_matrix(output, y_pred)
print(conf_mat)
plt.rcParams["figure.figsize"] = (3,3)
plt.figure()
plot_confusion_matrix(conf_mat, classes=['0','1'],
                     title='Confusion matrix')
plt.savefig("Confusion_Matrix.png", dpi=200)


#look specifically at which species are incorrectly and correctly predicted
species_list = list(df.index)
print('Incorrectly IDed  True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] != y_pred[i]:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
print('Correctly IDed   True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] == 1 and y_pred[i] == 1:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
    

header = list(input_ed.columns)

# define the model again to re-test on a held out dataset, usually have  subsample=0.9, colsample_bynode=0.8, 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
xtrain, xtest, ytrain, ytest=train_test_split(ml_input, output, test_size=0.10)
class_weights_training = compute_sample_weight('balanced', ytrain)
#sample_weight=class_weights_training
#model.fit(xtrain, ytrain) 
model.fit(xtrain, ytrain, sample_weight=class_weights_training) 

print('Mini_test')
ypred = model.predict(xtest)
cm = confusion_matrix(ytest,ypred)
total = sum(cm[0]) + sum(cm[1])
mis = cm[0][1] + cm[1][0]
print('Misclassification rate:', mis/total)

print(cm)

plt.figure()
plot_confusion_matrix(cm, classes=['0','1'],
                     title='Confusion matrix, without normalization')


#rebuild model yet again for the final big cross validation accuracy metrics with RepeatedStratifiedKFold
classes_weights = compute_sample_weight('balanced', output)
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
sample_weight=classes_weights
model.fit(ml_input, output, sample_weight=classes_weights)
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, ml_input, output, scoring='accuracy', cv=cv)
print(n_scores)
# report performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)), '\n')


#report feature importances in order of most important to least
importance_list = []
importances = list(model.feature_importances_)
o = 0
m = 0
top10 = []
for i in model.feature_importances_:
    importance_list.append([i, header[o]])
    o = o + 1
for i in importance_list:
    if i[0] == max(importances):
        print(i[1], '\t', i[0])
        if m < 10:
            top10.append(i[1])
            m = m + 1
            importances.remove(i[0])
        else:
            importances.remove(i[0])
    else:
        importance_list.append(i)

In [ ]:
#Figure S1: Predicting growth on galactose from environmental data--repeated for Xylose and Sucrose
import pandas as pd

#format input
input1= 'final_env_table_s4.csv'
input2='final_metabolic_table_s3.csv'

df1 = pd.read_csv(input1, index_col=0)
df2 = pd.read_csv(input2, index_col=0)
df = pd.concat([df1, df2.set_index(df1.index)], axis=1)
#df = pd.read_csv(input2, index_col=0)


#import modules needed
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBRFClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.utils import compute_sample_weight
import matplotlib.pyplot as plt
from collections import Counter

#define what we want to predict out of the dataset
predicting = 'C2_D.Galactose'
#drop NAs (species we didn't measure specialism/generalism) from the dataset
df = df.dropna(subset=[predicting])

#decide what other data you want to drop from the input, here I am dropping the predicted value
dropping = ['D.Glucose__Fermentation', 'D.Galactose__Fermentation', 'Maltose__Fermentation', 'Me_a.D.Glucoside__Fermentation', 'Sucrose__Fermentation', 'a_a.Trehalose__Fermentation', 'Melibiose__Fermentation', 'Lactose__Fermentation', 'Cellobiose__Fermentation', 'Melezitose__Fermentation', 'Raffinose__Fermentation', 'Inulin__Fermentation', 'Starch__Fermentation', 'D.Xylose__Fermentation', 'X_50__D.Glucose__O4_', 'X_60__D.Glucose__O5_', 'Starch_formation__M1_', 'Acid_acetic_production__M2_', 'Urea_hydrolysis__M3_', 'Diazonium_Blue_B_reaction__M4_', 'Methanol__assimilation', 'Ethanol__assimilation', 'C1_D.Glucose', 'C2_D.Galactose', 'C3_L.Sorbose', 'C4_D.Glucosamine', 'C5_D.Ribose', 'C6_D.Xylose', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C9_L.Rhamnose', 'C10_Sucrose', 'C11_Maltose', 'C12_a_a.Trehalose', 'C13_Me_a.D.Glucoside', 'C14_Cellobiose', 'C15_Salicin', 'C16_Arbutin', 'C17_Melibiose', 'C18_Lactose', 'C19_Raffinose', 'C20_Melezitose', 'C21_Inulin', 'C22_Starch', 'C23_Glycerol', 'C24_Erythritol', 'C25_Ribitol', 'C26_Xylitol', 'C27_L.Arabinitol', 'C28_D.Glucitol', 'C29_D.Mannitol', 'C30_Galactitol', 'C31_myo.Inositol', 'C32_D.Glucono.1_5.lactone', 'C33_2.Keto.D.Gluconate', 'C34_5.Keto.D.Gluconate', 'C35_D.Gluconate', 'C36_D.Glucuronate', 'C37_D.Galacturonate', 'C38_DL.Lactate', 'C39_Succinate', 'C40_Citrate', 'C43_Propane_1_2_diol', 'C44_Butane_2_3_diol', 'C45_Quinic_acid', 'C46_D.glucarate', 'C47_D.Galactonate', 'C48_Palatinose', 'C49_Levulinate', 'C50_L.Malic_acid', 'C51_L.Tartaric_acid', 'C52_D.Tartaric_acid', 'C53_meso.Tartaric_acid', 'C54_Galactaric_acid', 'C55_Uric_acid', 'C56_Gentobiose', 'C57_Ethylene_glycol', 'C58_Tween_40', 'C59_Tween_60', 'C60_Tween_80', 'N1_Nitrate', 'N2_Nitrite', 'N3_Ethylamine', 'N4_L.Lysine', 'N5_Cadaverine', 'N6_Creatine', 'N7_Creatinine', 'N8_Glucosamine', 'N9_Imidazole', 'N10_D.Tryptophan', 'N11_D.Proline', 'N12_Putrescine', 'V1_w_o_vitamins', 'V2_w_o_myo.Inositol', 'V3_w_o_Pantothenate', 'V4_w_o_Biotin', 'V5_w_o_Thiamin', 'V6_w_o_Biotin___Thiamin', 'V7_w_o_Pyridoxine', 'V8_w_o_Pyridoxine___Thiamin', 'V9_w_o_Niacin', 'V10_w_o_PABA', 'O1_Cycloheximide_0.01_', 'O2_Cycloheximide_0.1_', 'O3_Acetic_acid_1_', 'O6_10__NaCl', 'O7_16__NaCl', 'O8_Growth_at_pH_3', 'O9_Growth_at_pH_9.5', 'O10_Fluconazole', 'at_4_C', 'at_12_C', 'at_15_C', 'at_18_C', 'at_21_C', 'at_25_C', 'at_30_C', 'at_35_C', 'at_37_C', 'at_40_C', 'at_42_C', 'at_45_C']
input_ed = df.drop(dropping, axis=1)
#input_ed = df.drop(['C2_D.Galactose', 'D.Galactose__Fermentation'], axis=1)
ml_input = np.array(input_ed) #array of input
output = np.array(df[predicting]) #array of output
print(output)

#build your model 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)

#use cross_val_predict to test the model and build the big confusion matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(model, ml_input, output, cv=10)
conf_mat = confusion_matrix(output, y_pred)
print(conf_mat)
plt.rcParams["figure.figsize"] = (3,3)
plt.figure()
plot_confusion_matrix(conf_mat, classes=['0','1'],
                     title='Confusion matrix')
plt.savefig("Confusion_Matrix.png", dpi=200)


#look specifically at which species are incorrectly and correctly predicted
species_list = list(df.index)
print('Incorrectly IDed  True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] != y_pred[i]:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
print('Correctly IDed   True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] == 1 and y_pred[i] == 1:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
    

header = list(input_ed.columns)

# define the model again to re-test on a held out dataset, usually have  subsample=0.9, colsample_bynode=0.8, 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
xtrain, xtest, ytrain, ytest=train_test_split(ml_input, output, test_size=0.10)
class_weights_training = compute_sample_weight('balanced', ytrain)
#sample_weight=class_weights_training
#model.fit(xtrain, ytrain) 
model.fit(xtrain, ytrain, sample_weight=class_weights_training) 

print('Mini_test')
ypred = model.predict(xtest)
cm = confusion_matrix(ytest,ypred)
total = sum(cm[0]) + sum(cm[1])
mis = cm[0][1] + cm[1][0]
print('Misclassification rate:', mis/total)

print(cm)

plt.figure()
plot_confusion_matrix(cm, classes=['0','1'],
                     title='Confusion matrix, without normalization')


#rebuild model yet again for the final big cross validation accuracy metrics with RepeatedStratifiedKFold
classes_weights = compute_sample_weight('balanced', output)
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
sample_weight=classes_weights
model.fit(ml_input, output, sample_weight=classes_weights)
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, ml_input, output, scoring='accuracy', cv=cv)
print(n_scores)
# report performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)), '\n')


#report feature importances in order of most important to least
importance_list = []
importances = list(model.feature_importances_)
o = 0
m = 0
top10 = []
for i in model.feature_importances_:
    importance_list.append([i, header[o]])
    o = o + 1
for i in importance_list:
    if i[0] == max(importances):
        print(i[1], '\t', i[0])
        if m < 10:
            top10.append(i[1])
            m = m + 1
            importances.remove(i[0])
        else:
            importances.remove(i[0])
    else:
        importance_list.append(i)

In [ ]:
#Figure 5: Predicting growth on galactose from just the GAL genes
import pandas as pd

#format input
input1= 'GAL_genes.csv'
input2='final_metabolic_table_s3.csv'

df1 = pd.read_csv(input1, index_col=0)
df2 = pd.read_csv(input2, index_col=0)
df = pd.concat([df1, df2.set_index(df1.index)], axis=1)
#df = pd.read_csv(input2, index_col=0)


#import modules needed
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBRFClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.utils import compute_sample_weight
import matplotlib.pyplot as plt
from collections import Counter

#define what we want to predict out of the dataset
predicting = 'C2_D.Galactose'
#drop NAs (species we didn't measure specialism/generalism) from the dataset
df = df.dropna(subset=[predicting])

#decide what other data you want to drop from the input, here I am dropping the predicted value
dropping = ['D.Glucose__Fermentation', 'D.Galactose__Fermentation', 'Maltose__Fermentation', 'Me_a.D.Glucoside__Fermentation', 'Sucrose__Fermentation', 'a_a.Trehalose__Fermentation', 'Melibiose__Fermentation', 'Lactose__Fermentation', 'Cellobiose__Fermentation', 'Melezitose__Fermentation', 'Raffinose__Fermentation', 'Inulin__Fermentation', 'Starch__Fermentation', 'D.Xylose__Fermentation', 'X_50__D.Glucose__O4_', 'X_60__D.Glucose__O5_', 'Starch_formation__M1_', 'Acid_acetic_production__M2_', 'Urea_hydrolysis__M3_', 'Diazonium_Blue_B_reaction__M4_', 'Methanol__assimilation', 'Ethanol__assimilation', 'C1_D.Glucose', 'C2_D.Galactose', 'C3_L.Sorbose', 'C4_D.Glucosamine', 'C5_D.Ribose', 'C6_D.Xylose', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C9_L.Rhamnose', 'C10_Sucrose', 'C11_Maltose', 'C12_a_a.Trehalose', 'C13_Me_a.D.Glucoside', 'C14_Cellobiose', 'C15_Salicin', 'C16_Arbutin', 'C17_Melibiose', 'C18_Lactose', 'C19_Raffinose', 'C20_Melezitose', 'C21_Inulin', 'C22_Starch', 'C23_Glycerol', 'C24_Erythritol', 'C25_Ribitol', 'C26_Xylitol', 'C27_L.Arabinitol', 'C28_D.Glucitol', 'C29_D.Mannitol', 'C30_Galactitol', 'C31_myo.Inositol', 'C32_D.Glucono.1_5.lactone', 'C33_2.Keto.D.Gluconate', 'C34_5.Keto.D.Gluconate', 'C35_D.Gluconate', 'C36_D.Glucuronate', 'C37_D.Galacturonate', 'C38_DL.Lactate', 'C39_Succinate', 'C40_Citrate', 'C43_Propane_1_2_diol', 'C44_Butane_2_3_diol', 'C45_Quinic_acid', 'C46_D.glucarate', 'C47_D.Galactonate', 'C48_Palatinose', 'C49_Levulinate', 'C50_L.Malic_acid', 'C51_L.Tartaric_acid', 'C52_D.Tartaric_acid', 'C53_meso.Tartaric_acid', 'C54_Galactaric_acid', 'C55_Uric_acid', 'C56_Gentobiose', 'C57_Ethylene_glycol', 'C58_Tween_40', 'C59_Tween_60', 'C60_Tween_80', 'N1_Nitrate', 'N2_Nitrite', 'N3_Ethylamine', 'N4_L.Lysine', 'N5_Cadaverine', 'N6_Creatine', 'N7_Creatinine', 'N8_Glucosamine', 'N9_Imidazole', 'N10_D.Tryptophan', 'N11_D.Proline', 'N12_Putrescine', 'V1_w_o_vitamins', 'V2_w_o_myo.Inositol', 'V3_w_o_Pantothenate', 'V4_w_o_Biotin', 'V5_w_o_Thiamin', 'V6_w_o_Biotin___Thiamin', 'V7_w_o_Pyridoxine', 'V8_w_o_Pyridoxine___Thiamin', 'V9_w_o_Niacin', 'V10_w_o_PABA', 'O1_Cycloheximide_0.01_', 'O2_Cycloheximide_0.1_', 'O3_Acetic_acid_1_', 'O6_10__NaCl', 'O7_16__NaCl', 'O8_Growth_at_pH_3', 'O9_Growth_at_pH_9.5', 'O10_Fluconazole', 'at_4_C', 'at_12_C', 'at_15_C', 'at_18_C', 'at_21_C', 'at_25_C', 'at_30_C', 'at_35_C', 'at_37_C', 'at_40_C', 'at_42_C', 'at_45_C']
input_ed = df.drop(dropping, axis=1)
#input_ed = df.drop(['C2_D.Galactose', 'D.Galactose__Fermentation'], axis=1)
ml_input = np.array(input_ed) #array of input
output = np.array(df[predicting]) #array of output
print(output)

#build your model 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)

#use cross_val_predict to test the model and build the big confusion matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(model, ml_input, output, cv=10)
conf_mat = confusion_matrix(output, y_pred)
print(conf_mat)
plt.rcParams["figure.figsize"] = (3,3)
plt.figure()
plot_confusion_matrix(conf_mat, classes=['0','1'],
                     title='Confusion matrix')
plt.savefig("Confusion_Matrix.png", dpi=200)


#look specifically at which species are incorrectly and correctly predicted
species_list = list(df.index)
print('Incorrectly IDed  True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] != y_pred[i]:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
print('Correctly IDed   True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] == 1 and y_pred[i] == 1:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
    

header = list(input_ed.columns)

# define the model again to re-test on a held out dataset, usually have  subsample=0.9, colsample_bynode=0.8, 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
xtrain, xtest, ytrain, ytest=train_test_split(ml_input, output, test_size=0.10)
class_weights_training = compute_sample_weight('balanced', ytrain)
#sample_weight=class_weights_training
#model.fit(xtrain, ytrain) 
model.fit(xtrain, ytrain, sample_weight=class_weights_training) 

print('Mini_test')
ypred = model.predict(xtest)
cm = confusion_matrix(ytest,ypred)
total = sum(cm[0]) + sum(cm[1])
mis = cm[0][1] + cm[1][0]
print('Misclassification rate:', mis/total)

print(cm)

plt.figure()
plot_confusion_matrix(cm, classes=['0','1'],
                     title='Confusion matrix, without normalization')


#rebuild model yet again for the final big cross validation accuracy metrics with RepeatedStratifiedKFold
classes_weights = compute_sample_weight('balanced', output)
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
sample_weight=classes_weights
model.fit(ml_input, output, sample_weight=classes_weights)
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, ml_input, output, scoring='accuracy', cv=cv)
print(n_scores)
# report performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)), '\n')


#report feature importances in order of most important to least
importance_list = []
importances = list(model.feature_importances_)
o = 0
m = 0
top10 = []
for i in model.feature_importances_:
    importance_list.append([i, header[o]])
    o = o + 1
for i in importance_list:
    if i[0] == max(importances):
        print(i[1], '\t', i[0])
        if m < 10:
            top10.append(i[1])
            m = m + 1
            importances.remove(i[0])
        else:
            importances.remove(i[0])
    else:
        importance_list.append(i)

In [ ]:
#Figure 5: Predicting growth on galactose from GAL genes and metabolic data
import pandas as pd

#format input
input1= 'GAL_genes.csv'
input2='final_metabolic_table_s3.csv'

df1 = pd.read_csv(input1, index_col=0)
df2 = pd.read_csv(input2, index_col=0)
df = pd.concat([df1, df2.set_index(df1.index)], axis=1)
#df = pd.read_csv(input2, index_col=0)


#import modules needed
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBRFClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.utils import compute_sample_weight
import matplotlib.pyplot as plt
from collections import Counter

#define what we want to predict out of the dataset
predicting = 'C2_D.Galactose'
#drop NAs (species we didn't measure specialism/generalism) from the dataset
df = df.dropna(subset=[predicting])

#decide what other data you want to drop from the input, here I am dropping the predicted value
dropping = ['C2_D.Galactose', 'D.Galactose__Fermentation']
input_ed = df.drop(dropping, axis=1)
#input_ed = df.drop(['C2_D.Galactose', 'D.Galactose__Fermentation'], axis=1)
ml_input = np.array(input_ed) #array of input
output = np.array(df[predicting]) #array of output
print(output)

#build your model 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)

#use cross_val_predict to test the model and build the big confusion matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(model, ml_input, output, cv=10)
conf_mat = confusion_matrix(output, y_pred)
print(conf_mat)
plt.rcParams["figure.figsize"] = (3,3)
plt.figure()
plot_confusion_matrix(conf_mat, classes=['0','1'],
                     title='Confusion matrix')
plt.savefig("Confusion_Matrix.png", dpi=200)


#look specifically at which species are incorrectly and correctly predicted
species_list = list(df.index)
print('Incorrectly IDed  True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] != y_pred[i]:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
print('Correctly IDed   True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] == 1 and y_pred[i] == 1:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
    

header = list(input_ed.columns)

# define the model again to re-test on a held out dataset, usually have  subsample=0.9, colsample_bynode=0.8, 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
xtrain, xtest, ytrain, ytest=train_test_split(ml_input, output, test_size=0.10)
class_weights_training = compute_sample_weight('balanced', ytrain)
#sample_weight=class_weights_training
#model.fit(xtrain, ytrain) 
model.fit(xtrain, ytrain, sample_weight=class_weights_training) 

print('Mini_test')
ypred = model.predict(xtest)
cm = confusion_matrix(ytest,ypred)
total = sum(cm[0]) + sum(cm[1])
mis = cm[0][1] + cm[1][0]
print('Misclassification rate:', mis/total)

print(cm)

plt.figure()
plot_confusion_matrix(cm, classes=['0','1'],
                     title='Confusion matrix, without normalization')


#rebuild model yet again for the final big cross validation accuracy metrics with RepeatedStratifiedKFold
classes_weights = compute_sample_weight('balanced', output)
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
sample_weight=classes_weights
model.fit(ml_input, output, sample_weight=classes_weights)
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, ml_input, output, scoring='accuracy', cv=cv)
print(n_scores)
# report performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)), '\n')


#report feature importances in order of most important to least
importance_list = []
importances = list(model.feature_importances_)
o = 0
m = 0
top10 = []
for i in model.feature_importances_:
    importance_list.append([i, header[o]])
    o = o + 1
for i in importance_list:
    if i[0] == max(importances):
        print(i[1], '\t', i[0])
        if m < 10:
            top10.append(i[1])
            m = m + 1
            importances.remove(i[0])
        else:
            importances.remove(i[0])
    else:
        importance_list.append(i)

In [ ]:
#Figure 6: Predicting growth on galactose from GAL genes and galactitol
import pandas as pd

#format input
input1= 'GAL_genes.csv'
input2='final_metabolic_table_s3.csv'

df1 = pd.read_csv(input1, index_col=0)
df2 = pd.read_csv(input2, index_col=0)
df = pd.concat([df1, df2.set_index(df1.index)], axis=1)
#df = pd.read_csv(input2, index_col=0)


#import modules needed
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBRFClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.utils import compute_sample_weight
import matplotlib.pyplot as plt
from collections import Counter

#define what we want to predict out of the dataset
predicting = 'C2_D.Galactose'
#drop NAs (species we didn't measure specialism/generalism) from the dataset
df = df.dropna(subset=[predicting])

#decide what other data you want to drop from the input, here I am dropping the predicted value
dropping = ['D.Glucose__Fermentation', 'D.Galactose__Fermentation', 'Maltose__Fermentation', 'Me_a.D.Glucoside__Fermentation', 'Sucrose__Fermentation', 'a_a.Trehalose__Fermentation', 'Melibiose__Fermentation', 'Lactose__Fermentation', 'Cellobiose__Fermentation', 'Melezitose__Fermentation', 'Raffinose__Fermentation', 'Inulin__Fermentation', 'Starch__Fermentation', 'D.Xylose__Fermentation', 'X_50__D.Glucose__O4_', 'X_60__D.Glucose__O5_', 'Starch_formation__M1_', 'Acid_acetic_production__M2_', 'Urea_hydrolysis__M3_', 'Diazonium_Blue_B_reaction__M4_', 'Methanol__assimilation', 'Ethanol__assimilation', 'C1_D.Glucose', 'C2_D.Galactose', 'C3_L.Sorbose', 'C4_D.Glucosamine', 'C5_D.Ribose', 'C6_D.Xylose', 'C7_L.Arabinose', 'C8_D.Arabinose', 'C9_L.Rhamnose', 'C10_Sucrose', 'C11_Maltose', 'C12_a_a.Trehalose', 'C13_Me_a.D.Glucoside', 'C14_Cellobiose', 'C15_Salicin', 'C16_Arbutin', 'C17_Melibiose', 'C18_Lactose', 'C19_Raffinose', 'C20_Melezitose', 'C21_Inulin', 'C22_Starch', 'C23_Glycerol', 'C24_Erythritol', 'C25_Ribitol', 'C26_Xylitol', 'C27_L.Arabinitol', 'C28_D.Glucitol', 'C29_D.Mannitol', 'C31_myo.Inositol', 'C32_D.Glucono.1_5.lactone', 'C33_2.Keto.D.Gluconate', 'C34_5.Keto.D.Gluconate', 'C35_D.Gluconate', 'C36_D.Glucuronate', 'C37_D.Galacturonate', 'C38_DL.Lactate', 'C39_Succinate', 'C40_Citrate', 'C43_Propane_1_2_diol', 'C44_Butane_2_3_diol', 'C45_Quinic_acid', 'C46_D.glucarate', 'C47_D.Galactonate', 'C48_Palatinose', 'C49_Levulinate', 'C50_L.Malic_acid', 'C51_L.Tartaric_acid', 'C52_D.Tartaric_acid', 'C53_meso.Tartaric_acid', 'C54_Galactaric_acid', 'C55_Uric_acid', 'C56_Gentobiose', 'C57_Ethylene_glycol', 'C58_Tween_40', 'C59_Tween_60', 'C60_Tween_80', 'N1_Nitrate', 'N2_Nitrite', 'N3_Ethylamine', 'N4_L.Lysine', 'N5_Cadaverine', 'N6_Creatine', 'N7_Creatinine', 'N8_Glucosamine', 'N9_Imidazole', 'N10_D.Tryptophan', 'N11_D.Proline', 'N12_Putrescine', 'V1_w_o_vitamins', 'V2_w_o_myo.Inositol', 'V3_w_o_Pantothenate', 'V4_w_o_Biotin', 'V5_w_o_Thiamin', 'V6_w_o_Biotin___Thiamin', 'V7_w_o_Pyridoxine', 'V8_w_o_Pyridoxine___Thiamin', 'V9_w_o_Niacin', 'V10_w_o_PABA', 'O1_Cycloheximide_0.01_', 'O2_Cycloheximide_0.1_', 'O3_Acetic_acid_1_', 'O6_10__NaCl', 'O7_16__NaCl', 'O8_Growth_at_pH_3', 'O9_Growth_at_pH_9.5', 'O10_Fluconazole', 'at_4_C', 'at_12_C', 'at_15_C', 'at_18_C', 'at_21_C', 'at_25_C', 'at_30_C', 'at_35_C', 'at_37_C', 'at_40_C', 'at_42_C', 'at_45_C']
input_ed = df.drop(dropping, axis=1)
#input_ed = df.drop(['C2_D.Galactose', 'D.Galactose__Fermentation'], axis=1)
ml_input = np.array(input_ed) #array of input
output = np.array(df[predicting]) #array of output
print(output)

#build your model 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)

#use cross_val_predict to test the model and build the big confusion matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(model, ml_input, output, cv=10)
conf_mat = confusion_matrix(output, y_pred)
print(conf_mat)
plt.rcParams["figure.figsize"] = (3,3)
plt.figure()
plot_confusion_matrix(conf_mat, classes=['0','1'],
                     title='Confusion matrix')
plt.savefig("Confusion_Matrix.png", dpi=200)


#look specifically at which species are incorrectly and correctly predicted
species_list = list(df.index)
print('Incorrectly IDed  True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] != y_pred[i]:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
print('Correctly IDed   True_Value  Predicted_Value')
for i in range(0, len(species_list)):
    if output[i] == 1 and y_pred[i] == 1:
        print(species_list[i], output[i], y_pred[i])
    else:
        pass
    

header = list(input_ed.columns)

# define the model again to re-test on a held out dataset, usually have  subsample=0.9, colsample_bynode=0.8, 
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
xtrain, xtest, ytrain, ytest=train_test_split(ml_input, output, test_size=0.10)
class_weights_training = compute_sample_weight('balanced', ytrain)
#sample_weight=class_weights_training
#model.fit(xtrain, ytrain) 
model.fit(xtrain, ytrain, sample_weight=class_weights_training) 

print('Mini_test')
ypred = model.predict(xtest)
cm = confusion_matrix(ytest,ypred)
total = sum(cm[0]) + sum(cm[1])
mis = cm[0][1] + cm[1][0]
print('Misclassification rate:', mis/total)

print(cm)

plt.figure()
plot_confusion_matrix(cm, classes=['0','1'],
                     title='Confusion matrix, without normalization')


#rebuild model yet again for the final big cross validation accuracy metrics with RepeatedStratifiedKFold
classes_weights = compute_sample_weight('balanced', output)
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs = 8)
sample_weight=classes_weights
model.fit(ml_input, output, sample_weight=classes_weights)
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, ml_input, output, scoring='accuracy', cv=cv)
print(n_scores)
# report performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)), '\n')


#report feature importances in order of most important to least
importance_list = []
importances = list(model.feature_importances_)
o = 0
m = 0
top10 = []
for i in model.feature_importances_:
    importance_list.append([i, header[o]])
    o = o + 1
for i in importance_list:
    if i[0] == max(importances):
        print(i[1], '\t', i[0])
        if m < 10:
            top10.append(i[1])
            m = m + 1
            importances.remove(i[0])
        else:
            importances.remove(i[0])
    else:
        importance_list.append(i)

In [ ]:
#ROC AUC code: plug in input files, predicting variable, and dropping variables.
#example here is growth on galactose predicted from metabolic data
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBRFClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.utils import compute_sample_weight
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics as metrics


df = pd.read_csv('final_metabolic_table_s3.csv', index_col=0)
predicting = 'C2_D.Galactose'
df = df.dropna(subset=[predicting])
input_ed = df.drop(['C2_D.Galactose', 'D.Galactose__Fermentation'], axis=1)
X = np.array(input_ed) #array of input
y = np.array(df[predicting])


from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve

# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=10)
model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss', n_jobs=8)
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
tprs = []
base_fpr = np.linspace(0, 1, 101)

    
plt.figure(figsize=(5, 5))
plt.axes().set_aspect('equal', 'datalim')

for i, (train, test) in enumerate(cv.split(X, y)):
    model = XGBRFClassifier(max_depth=12, n_estimators=100, use_label_encoder =False,  eval_metric='mlogloss',  n_jobs=8)
    model.fit(X[train], y[train])
    y_score = model.predict_proba(X[test])
    fpr, tpr, _ = roc_curve(y[test], y_score[:, 1])
    roc_auc = metrics.auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, 'b', alpha=0.15)
    tpr = np.interp(base_fpr, fpr, tpr)
    tpr[0] = 0.0
    tprs.append(tpr)

tprs = np.array(tprs)
mean_tprs = tprs.mean(axis=0)
aucs = np.array(aucs)
mean_auc = aucs.mean(axis=0)
std = tprs.std(axis=0)

tprs_upper = np.minimum(mean_tprs + std, 1)
tprs_lower = mean_tprs - std


plt.plot(base_fpr, mean_tprs, 'b', label = 'Mean ROC (AUC = %0.2f)' % mean_auc )
plt.legend(loc = 'lower right')
plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3, label=r"$\pm$ 1 std. dev.")
plt.legend(loc = 'lower right')

plt.plot([0, 1], [0, 1],'r--', label="Chance")
plt.legend(loc = 'lower right')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig("gen_spec.png", dpi=500)
plt.show()